# Common Join Quirks

We have now covered the most common types of join and therefore it's worth taking a moment to highlight some times when these joins can perform in what seems to be a strange manner.

However, this is always by design and easy to explain and avoid once you are comfortable using them.

#### Missing LEFT JOIN Records

So what do we mean by missing records? This sounds scary and, to be honest, when you first see it happen it can throw you completely. Don't worry though, it's perfectly normal to see and just as easy to fix once you realise what has occured.

So what do we mean by this?

Well let's take a look at our generic RIGHT JOIN code to start with:

In [ ]:
select e.employeeID, e.firstName, e.lastName,
		s.employeeID, s.orderDate
from JupyterDatabase.hr.employeeSmall as e
right join JupyterDatabase.sales.salesOrderHeaderSmall as s
on e.employeeID = s.employeeID;

When looking at the output of this query we can see that employeeID 45 does not appear in our hr.employeeSmall table.

Looking more closely at the orders for employeeID 45 we can see that there were two that were made in November 2019.

So now let's consider this very contrived query requirement:

-   Show all Employees
    -   First and Last names
-   Only show orders if they were placed in November 2019

As mentioned, this is contrived but also designed to prove a point so let's have a closer look.

Firstly we know that we wish to see all employees and only orders where they exist based on a criteria. Therefore we should immediately be thinking LEFT JOIN (or RIGHT JOIN depending how your head works):

In [ ]:
select e.employeeID, e.firstName, e.lastName,
		s.employeeID, s.orderDate
from JupyterDatabase.hr.employeeSmall as e
left join JupyterDatabase.sales.salesOrderHeaderSmall as s
on e.employeeID = s.employeeID;

So far, so good.

Now we need to only show orders if they were placed in November 2019. Easy enough?

In [ ]:
select e.employeeID, e.firstName, e.lastName,
		s.employeeID, s.orderDate
from JupyterDatabase.hr.employeeSmall as e
left join JupyterDatabase.sales.salesOrderHeaderSmall as s
on e.employeeID = s.employeeID
where month(s.orderDate) = 11;

Hang on.  What went wrong?

We wrote a LEFT JOIN and therefore we should see the entire of the FROM table and only what matches in the JOIN table, so why do we have no results at all?

This is the quirk we needed to raise and discuss.

If you remember the processing order SQL Server will process the FROM and JOIN clauses prior to the WHERE clause.

Let's repeat SQL Server's actions:

In [ ]:
select e.employeeID, e.firstName, e.lastName,
		s.employeeID, s.orderDate
from JupyterDatabase.hr.employeeSmall as e
left join JupyterDatabase.sales.salesOrderHeaderSmall as s
on e.employeeID = s.employeeID;

Performing our LEFT JOIN on its own we can see that based on the output of this query there are no records available with month(orderDate) 11.

Therefore, noting we are following the processing order, we now have a table of results and need to fulfil the next step, applying the WHERE filter to our resulting table.

As we have no records in our table with an month(orderDate) 11 the filter now returns zero records.

This is a problem as it is not the data that we were expecting to see.

#### Solution

The method we need to adopt in this situation is to use the ON clause in our favour.

If we say to SQL Server 'In the ON clause, please apply this filter' it will do this as part of the JOIN and therefore will filter the raw table and then use that in the join.

To detail this a little better, by placing the filter in the ON clause we are effectively asking SQL Server to do the following:

In [ ]:
select e.employeeID, e.firstName, e.lastName,
		s.employeeID, s.orderDate
from JupyterDatabase.hr.employeeSmall as e
left join
(
        select *
        from JupyterDatabase.sales.salesOrderHeaderSmall
        where month(orderDate) = 11
) as s
on e.employeeID = s.employeeID;

Ie. We need to filter the table that we are joining to and NOT filter the post-join results.

To do this we can place the filter in the ON clause as this acts on the table at the time of the JOIN and not once the JOIN has already taken place:

In [ ]:
select e.employeeID, e.firstName, e.lastName,
		s.employeeID, s.orderDate
from JupyterDatabase.hr.employeeSmall as e
left join JupyterDatabase.sales.salesOrderHeaderSmall as s
on e.employeeID = s.employeeID
and month(s.orderDate) = 11;

As you can see, this gives us the result we are looking for (as does the bracketed version which is called a derived query, but we will cover that later, for the moment this is a cleaner approach to this issue).

#### ON versus WHERE

As you can see from the above, both the ON and WHERE can be used to filter a dataset and in some cases these are interchangeable.

However, as the above also shows, there are cases when these are definitely not interchangeable and strange results can happen if you get this inadvertently incorrect.  Therefore it is always recommended that JOIN conditions are placed in the ON clause and query filters are placed in the WHERE clause.